In [60]:
import pandas as pd
import numpy as np
import folium
import math
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from math import cos, asin, sqrt, pi
from folium.plugins import HeatMap
from folium import plugins
import sys

In [61]:
def get_similar(df):
    df['amenity'] = df['amenity'].replace(['parking_entrance'], 'parking')
    df['amenity'] = df['amenity'].replace(['pub'], 'bar')
    df['amenity'] = df['amenity'].replace(['childcare'], 'kindergarten')
    df['amenity'] = df['amenity'].replace(['nightclub'], 'bar')
    df['amenity'] = df['amenity'].replace(['gambling'], 'casino')
    df['amenity'] = df['amenity'].replace(['atm;bank'], 'bank')
    df['amenity'] = df['amenity'].replace(['motorcycle_parking'], 'parking')
    df['amenity'] = df['amenity'].replace(['drinking_water'], 'fountain')
    df['amenity'] = df['amenity'].replace(['doctors'], 'hospital')
    df['amenity'] = df['amenity'].replace(['storage'], 'storage_rental')
    df['amenity'] = df['amenity'].replace(['internet_cafe'], 'cafe')
    df['amenity'] = df['amenity'].replace(['chiropractor'], 'hospital')
    df['amenity'] = df['amenity'].replace(['post_depot'], 'post_office')
    df['amenity'] = df['amenity'].replace(['Pharmacy'], 'pharmacy')
    return df

In [62]:
def map_scatter_plot(data_file):
    # read file
    data = pd.read_csv(data_file, index_col=0)

    data_map = folium.Map(location=[49.284808779798745, -123.12244539679432], zoom_start=10)

    # add every data point to the map
    for index, row in data.iterrows():
        folium.Circle(
            radius=10,
            location=[row['lat'], row['lon']],
            color='blue',
        ).add_to(data_map)
    return data_map
        

In [63]:
def main():
    raw_data= pd.read_json("amenities-vancouver.json.gz", lines=True)
    raw_data = raw_data[raw_data['name'].notna()]
    distinct_data =raw_data['amenity'].unique()
    data = get_similar(raw_data)
    filt = ['restaurant']
    restaurant = data[data['amenity'].isin(filt) == True]

    # incase of data missing restaurant name
    restaurant = restaurant.dropna(subset=['name']).reset_index(drop = True)
    # get all restaurant in vancouver
    restaurant.to_csv('all_restaurant.csv')
    all_restaurant = pd.read_csv("all_restaurant.csv")
    # remove duplicate information
    all_restaurant.drop_duplicates()
    
    chain_restaurant = all_restaurant[all_restaurant.duplicated(subset=['name'])==True].reset_index(drop = True)
    chain_restaurant.to_csv('chain_restaurant.csv')
    chain_restaurant = pd.read_csv("chain_restaurant.csv")
    chain_restaurant = chain_restaurant[['lat','lon','name']] 
    all_restaurant = all_restaurant[['lat','lon','name']]
    data_map = folium.Map(location=[49.284808779798745, -123.12244539679432], zoom_start=10)
    location = []
    for index, row in all_restaurant.iterrows():
        location.append([row['lat'], row['lon']])
    data_map.add_child(plugins.HeatMap(location,blur=28))
    data_map.save(outfile= "all_restaurant.html")
    all_resturant_map = map_scatter_plot("all_restaurant.csv")
    all_resturant_map.save(outfile= "all_restaurant_plot.html")
    for index, row in chain_restaurant.iterrows():
        location.append([row['lat'], row['lon']])
    data_map.add_child(plugins.HeatMap(location,blur=28))

    data_map.save(outfile= "chain_restaurant.html")
    chain_resturant_map = map_scatter_plot("chain_restaurant.csv")
    chain_resturant_map.save(outfile= "chain_restaurant_plot.html")

In [64]:
if __name__=='__main__':
    main()
   
